In [1]:
import requests
from datetime import datetime
from collections import defaultdict

In [2]:
PROMETHEUS_HOST = "localhost:9090"
COUNTER_DEBUG = False

COUNTER_COUNT = 0
GAUGE_COUNT = 0

COUNTER_PASS = 0
GAUGE_PASS = 0

expected_metrics = defaultdict()
entries = []

# read and parse the answer file
expected_answers = open("PrometheusAnswersFirst.csv", "r")
lines = expected_answers.readlines()


def parse_label(label_str):
    label_arr = label_str.replace("{", "").replace("}", "").replace('\n', "").split(":")
    labels = {label_arr[0]: label_arr[1]}
    return labels


def request_and_check_single_value(metric_name):
    URL = "http://" + PROMETHEUS_HOST + "/api/v1/query?query=" + metric_name
    r = requests.get(URL)
    result = r.json()['data']['result']

    if result:
        actual_metric = result[0]['metric']
        actual_value = result[0]['value'][1]
        actual_name = actual_metric['__name__']
        return expected_metrics[actual_name]['value'] == actual_value
    return False


def request_and_check_label(metric_name):
    # send http request to Prometheus server
    URL = "http://" + PROMETHEUS_HOST + "/api/v1/query?query=" + metric_name
    r = requests.get(URL)
    result = r.json()['data']['result']
    if result:
        actual_metric = result[0]['metric']
        actual_name = actual_metric['__name__']
        key = "key" + actual_name.split('_')[0][4:]
        actual_label = result[0]['metric'][key]
        return expected_metrics[actual_name]['labels'][key] == actual_label
    return False

In [3]:
for line in lines:
    split_result = line.split("|")
    metric_type = split_result[0]
    others = split_result[-1].replace(" ", "")
    other_split = others.split(",")
    name = other_split[0]
    desc = other_split[1]
    labels_str = other_split[-1]

    metric_entry = {name: metric_type}
    entries.append(metric_entry)

    expected_metrics[name] = defaultdict()
    expected_metrics[name]['type'] = metric_type
    if metric_type == "counter":
        COUNTER_COUNT += 1
        val = split_result[1]
        expected_metrics[name]['value'] = val
        expected_metrics[name]['labels'] = parse_label(labels_str)
    elif metric_type == "gauge":
        GAUGE_COUNT += 1
        val = split_result[1]
        expected_metrics[name]['value'] = val
        expected_metrics[name]['labels'] = parse_label(labels_str)
    else:
        print("Unknown metric type")

In [ ]:
for metric_entry in entries:
    name = list(metric_entry.keys())[0]
    type = metric_entry[name]

    # send http request to Prometheus server
    # [{'metric': {'__name__': 'name2010_counter', 'instance': 'docker.for.mac.localhost:8080', 'job': 'test_prom_exporter', 'key2010': 'value2010'}, 'value': [1598204478.776, '-2']}]
    if request_and_check_single_value(name) and request_and_check_label(name):
        if type == "counter":
            COUNTER_PASS += 1
        elif type == "gauge":
            GAUGE_PASS += 1

print("Total number of Counter: " + str(COUNTER_COUNT))
print("Test passed Counter: " + str(COUNTER_PASS))
print("Counter test pass rate: " + str(COUNTER_PASS / COUNTER_COUNT * 100) + "%")

print("Total number of Gauge: " + str(COUNTER_COUNT))
print("Test passed Gauge: " + str(COUNTER_PASS))
print("Gauge test pass rate: " + str(COUNTER_PASS / COUNTER_COUNT * 100) + "%")